In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data_bayes.csv')
last_column = df.iloc[:, -1]
df

,Lp.,Obsługa,Jakość towaru,Obok drogi,Centrum,Decyzja
0,1,Dobra,Dobra,Nie,Nie,Strata
1,2,Dobra,Dobra,Nie,Tak,Strata
2,3,Bardzo dobra,Dobra,Nie,Nie,Zysk
3,4,Słaba,Super,Nie,Nie,Zysk
4,5,Słaba,Niska,Tak,Nie,Zysk
5,6,Słaba,Niska,Tak,Tak,Strata
6,7,Bardzo dobra,Niska,Tak,Tak,Strata
7,8,Dobra,Super,Nie,Nie,Zysk
8,9,Dobra,Niska,Tak,Nie,Zysk
9,10,Słaba,Super,Nie,Nie,Zysk


In [3]:
# we have two classes C1-strata(loss) C2-zysk(profit) we have to maximize the value P(X13|Ci)*P(Ci) for i-1,2


In [9]:
#Calculate the a priori probabilities P(C1) and P(C2) , which is the probability of loss and gain regardless of observation.
class_counts = df[last_column.name].value_counts()
total_cases = class_counts.sum()
decision_keys = df[df[last_column.name].notna()][last_column.name].unique().tolist()


def get_prclass(has_all_attr):
    p_c = {}
    for key in decision_keys:
        if has_all_attr:
            p_c[key] = (class_counts[key] / total_cases)
        else:
            p_c[key] = ((class_counts[key] + 1 )/ (total_cases + len(decision_keys)))
    return p_c


In [10]:
columns_key = df.columns.tolist()[1:-1] # Skip 'Lp.' and 'Decyzja'
columns_key

['Obsługa', 'Jakość towaru', 'Obok drogi', 'Centrum']

In [11]:
def get_freq(has_all_attr, d_key, attr, value):
    cp = {}

    # number of occurrences of values in the attribut
    unique_keys = len(df[df[attr].notna()][attr].unique().tolist())

    # dataframe with count of attributes for decision key 
    attrs_count = df[df[last_column.name] == d_key][attr].value_counts()

    # number of occurrences of the value in the attribute / check that all attributes are used
    attr_count = attrs_count.get(value, 0)

    if (attr_count == 0 and has_all_attr):
        return 0


    if has_all_attr:
        # P(count of attribut / count of types decision) = 
        cp = ( attrs_count / class_counts[d_key]).to_dict().get(value)
    else:
        # P(count of attribut + 1 / count of types decision +  count of types Ai)
        cp = (attr_count + 1) / (class_counts[d_key] + unique_keys )
   
    return cp

In [14]:
# calculation of probability for unknown values
# Multiply by the prior probabilities
# P(Xn | Ci)*P(Ci)

P={}
df_x = df[df[last_column.name].isna()]

for index, row in df_x.iterrows():
    id = row.iloc[0]
    P[id] = {}
    P[id]["has_all"] = True

    # check all attributes have class decision
    for c in columns_key:
        for d_key in decision_keys:
            attr_value = row[c]
            value = get_freq(True, d_key, c, attr_value)
            if value == 0:
                P[id]["has_all"] = False

    for c in columns_key:
        for d_key in decision_keys:
            attr_value = row[c]
            value = get_freq(P[id]["has_all"], d_key, c, attr_value)
            P[id][d_key] = P[id].get(d_key, 0.01) * value

In [16]:
for id in P:
    result = { "value": 0}
    for key in decision_keys:

        pc = get_prclass(P[id]["has_all"])
        result = {"value":P[id][key] * pc[key], "decision": key } if result["value"] < P[id][key] * pc[key] else result
        print(id, P[id][key] * pc[key])
    print(f"Row number #{id} probably has value \"{result["decision"]}\"\n")
    result = { "value": 0}

13 5.208333333333333e-05
13 3.9062499999999994e-05
Row number #13 probably has value "Strata"

14 4.859086491739552e-05
14 0.0002677685950413223
Row number #14 probably has value "Zysk"

